<a href="https://colab.research.google.com/github/pratik-poudel/datacrunch/blob/main/19th%20April%20Regression%20chain%20with%20crossval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import gc
# pd.options.display.float_format| = "{:.2f}".format
pd.options.display.max_columns = 500

import sklearn
from sklearn import *
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.utils.testing import all_estimators
from sklearn.base import RegressorMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.multioutput import RegressorChain


import requests
import time
from scipy import stats

import lightgbm as lgb
import xgboost as xgb

In [2]:
train_datalink_X = 'https://tournament.datacrunch.com/data/X_train.csv'  
train_datalink_y = 'https://tournament.datacrunch.com/data/y_train.csv'
hackathon_data_link = 'https://tournament.datacrunch.com/data/X_test.csv'

train = pd.read_csv(train_datalink_X)
target = pd.read_csv(train_datalink_y)
test = pd.read_csv(hackathon_data_link)

In [3]:
print(train.shape, test.shape)

(6538, 15) (3436, 15)


In [4]:
train.head()

,Moons,id,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13
0,0,0x9818876384b58d3c,0.50,1.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
1,0,0xf9aeb187b2dad4f,0.50,1.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
2,0,0x131db64190c7c8af,0.75,1.0,0.75,0.0,0.0,0.0,0.0,0.5,0.5,1.0,0.75,0.75,1.0
3,0,0xdbcbe7dd2f47b4ea,0.50,1.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.50,0.75,0.5
4,0,0x580e576eca4dc196,0.75,1.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0


In [5]:
target.head()

,target_r,target_g,target_b
0,0.25,0.25,0.25
1,0.00,0.00,0.00
2,0.50,0.50,0.50
3,0.00,0.00,0.00
4,0.75,1.00,0.75


In [6]:
feat= [f for f in train.columns if "Feature" in f]
print(feat)

['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7', 'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13']


# Function

In [ ]:
!pip install optuna

In [17]:
def scorer(y_test, y_pred):
    # print(y_test)
    true_r, pred_r = y_test['target_r'], y_pred[:, 0]
    true_g, pred_g = y_test['target_g'], y_pred[:, 1]
    true_b, pred_b = y_test['target_b'], y_pred[:, 2]

    score_r = (stats.spearmanr(true_r, pred_r))[0]
    score_g = (stats.spearmanr(true_r, pred_r))[0]
    score_b = (stats.spearmanr(true_r, pred_r))[0]
    # print(score_r, score_g, score_b)

    # print(score)
    # print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format((score_r + score_g + score_b)/3))
    return (score_r + score_g + score_b)/ 3 


n_split=5
n_repeat=2

my_scorer = make_scorer(scorer, greater_is_better=True)
cv = RepeatedKFold(n_splits=n_split, n_repeats=n_repeat, random_state=60)


In [20]:
import optuna

N_TRIALS = 200
def objective_hist(trial):    
    params = {
            'learning_rate' : trial.suggest_uniform('learning_rate', 0.0001, 0.99),
            'max_iter': trial.suggest_int('max_iter', 10, 100),
            'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 31, 500),
            'max_depth': trial.suggest_int('max_depth', 4, 32),
            'min_samples_leaf':trial.suggest_int('min_samples_leaf', 20, 100),
            'l2_regularization': trial.suggest_loguniform('l2_regularization', 1e-8, 1.0),
            'max_bins':trial.suggest_int('max_bins', 1, 255),
            # 'monotonic_cst' : trial.suggest_categorical('monotonic_cst',[-1, 1, 0])
            }

    model = sklearn.ensemble.HistGradientBoostingRegressor(**params, random_state=60)
    wrapper = RegressorChain(model)

    scores = cross_val_score(wrapper, train[feat], target, scoring=my_scorer, 
                             n_jobs=-1, 
                             cv= cv, 
                             )
    from google.colab import output
    output.clear()
    
    return np.mean(scores)
def make_study():
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_hist, n_trials=N_TRIALS, show_progress_bar= True)

    return study.best_params, study.best_value

In [21]:
hist_params , hist = make_study()

[I 2021-04-18 18:12:48,970] Trial 199 finished with value: 0.08506899296700518 and parameters: {'learning_rate': 0.13435499857205307, 'max_iter': 55, 'max_leaf_nodes': 429, 'max_depth': 17, 'min_samples_leaf': 84, 'l2_regularization': 0.07457672501927104, 'max_bins': 249}. Best is trial 85 with value: 0.08766255304581794.



In [23]:
hist_params, hist

({'l2_regularization': 0.004091806042793742,
  'learning_rate': 0.0794741489718396,
  'max_bins': 226,
  'max_depth': 15,
  'max_iter': 71,
  'max_leaf_nodes': 423,
  'min_samples_leaf': 83},
 0.08766255304581794)

In [29]:
def objective_xgb(trial):
    # XGBoost parameters
    params = {
        "verbosity": 0,  # 0 (silent) - 3 (debug)
        "objective": "reg:squarederror",
        "n_estimators": 100,
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.05),
        "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.2, 0.6),
        "subsample": trial.suggest_loguniform("subsample", 0.4, 0.8),
        "alpha": trial.suggest_loguniform("alpha", 0.01, 10.0),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "gamma": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 10, 1000),
        "seed": 60,
        "n_jobs": -1,
    }

    model = xgb.XGBRegressor(**params, random_state=60)
    wrapper = RegressorChain(model)
    scores = cross_val_score(wrapper, train[feat], target, scoring=my_scorer, 
                             n_jobs=-1, 
                             cv= cv, 
                             )
    from google.colab import output
    output.clear()
    
    return np.mean(scores)
def make_study():
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_xgb, n_trials=N_TRIALS, show_progress_bar= True)

    return study.best_params, study.best_value

xgb_params , xgb = make_study()
   

[I 2021-04-18 19:01:12,327] Trial 199 finished with value: 0.08757614793135002 and parameters: {'max_depth': 11, 'learning_rate': 0.018446358465857973, 'colsample_bytree': 0.5683874692054515, 'subsample': 0.48852598527394325, 'alpha': 1.4904580042665625, 'lambda': 2.9206316832532792e-05, 'min_child_weight': 15.698529816016197}. Best is trial 184 with value: 0.09877967942836713.



In [31]:
xgb_params, lars_params, hist_params

({'alpha': 1.7306624641862671,
  'colsample_bytree': 0.5617715269369429,
  'lambda': 3.118325743297885e-05,
  'learning_rate': 0.017640977280002332,
  'max_depth': 11,
  'min_child_weight': 19.44024843008057,
  'subsample': 0.46751165458207533},
 {'l2_regularization': 2.2954895330867107e-06,
  'learning_rate': 0.12811384163378298,
  'max_bins': 91,
  'max_depth': 5,
  'max_iter': 65,
  'max_leaf_nodes': 131,
  'min_samples_leaf': 71},
 {'l2_regularization': 0.004091806042793742,
  'learning_rate': 0.0794741489718396,
  'max_bins': 226,
  'max_depth': 15,
  'max_iter': 71,
  'max_leaf_nodes': 423,
  'min_samples_leaf': 83})

In [41]:
from xgboost import XGBRegressor
model_xgb =XGBRegressor(**xgb_params, random_state=60)
model_hist = sklearn.ensemble.HistGradientBoostingRegressor(**hist_params)

In [46]:
train[feat]

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13
0,0.50,1.00,0.75,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.50,1.00,0.75,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.75,1.00,0.75,0.0,0.0,0.00,0.00,0.50,0.50,1.00,0.75,0.75,1.00
3,0.50,1.00,0.75,0.0,0.0,0.00,0.00,0.00,0.00,0.50,0.50,0.75,0.50
4,0.75,1.00,0.75,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6533,0.25,0.50,0.50,1.0,1.0,0.75,0.75,0.75,0.75,0.00,0.00,0.00,0.00
6534,0.00,0.25,0.25,1.0,1.0,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.00
6535,0.50,0.50,0.50,1.0,1.0,1.00,1.00,1.00,1.00,0.25,0.50,0.75,0.50
6536,0.25,0.75,1.00,1.0,1.0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00


In [51]:
cv.split(X)

<generator object _RepeatedSplits.split at 0x7f5d503f1b50>

In [50]:
models = []
X = train[feat]
for train_index,test_index in cv.split(X):
    train_features = X.loc[train_index]
    train_target = X.loc[train_index]
    
    test_features = X.loc[test_index]
    test_target = X.loc[test_index]
    model = model_xgb
    wrapper = RegressorChain(model)
    wrapper.fit(train_features, train_target)

    models.append(model)
    gc.collect()

[19:49:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

KeyboardInterrupt: ignored

In [ ]:
API_KEY = "0GIskIC9Rd1I8sMw6nStutbIbMx7AaZzAyNfuMOVAVWgJtNa1IVQHZAD3WPj" # <- HERE

r = requests.post("https://tournament.datacrunch.com/api/submission",
    files = {
        "file": ("x", prediction.to_csv().encode('ascii'))
    },
    data = {
        "apiKey": API_KEY
    },
)

if r.status_code == 200:
    print("Submission submitted :)")
elif r.status_code == 423:
    print("ERR: Submissions are close")
    print("You can only submit during rounds eg: Friday 7pm GMT+1 to Sunday midnight GMT+1.")
    print("Or the server is currently crunching the submitted files, please wait some time before retrying.")
elif r.status_code == 422:
    print("ERR: API Key is missing or empty")
    print("Did you forget to fill the API_KEY variable?")
elif r.status_code == 404:
    print("ERR: Unknown API Key")
    print("You should check that the provided API key is valid and is the same as the one you've received by email.")
elif r.status_code == 400:
    print("ERR: The file must not be empty")
    print("You have send a empty file.")
elif r.status_code == 401:
    print("ERR: Your email hasn't been verified")
    print("Please verify your email or contact a cruncher.")
elif r.status_code == 429:
    print("ERR: Too many submissions")
else:
    print("ERR: Server returned: " + str(r.status_code))
    print("Ouch! It seems that we were not expecting this kind of result from the server, if the probleme persist, contact a cruncher.")

Submission submitted :)


In [ ]:
from sklearn.model_selection import  cross_val_predict

TypeError: ignored

In [ ]:
train

,Moons,id,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13
0,0,0x9818876384b58d3c,0.50,1.00,0.75,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0,0xf9aeb187b2dad4f,0.50,1.00,0.75,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0,0x131db64190c7c8af,0.75,1.00,0.75,0.0,0.0,0.00,0.00,0.50,0.50,1.00,0.75,0.75,1.00
3,0,0xdbcbe7dd2f47b4ea,0.50,1.00,0.75,0.0,0.0,0.00,0.00,0.00,0.00,0.50,0.50,0.75,0.50
4,0,0x580e576eca4dc196,0.75,1.00,0.75,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6533,13,0xee1509f813ae5a29,0.25,0.50,0.50,1.0,1.0,0.75,0.75,0.75,0.75,0.00,0.00,0.00,0.00
6534,13,0x5be7f5f0950653aa,0.00,0.25,0.25,1.0,1.0,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.00
6535,13,0x69cf43587a77375b,0.50,0.50,0.50,1.0,1.0,1.00,1.00,1.00,1.00,0.25,0.50,0.75,0.50
6536,13,0xd3618b1bcfac1137,0.25,0.75,1.00,1.0,1.0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00
